# V7 Labs to COCO

See

* https://github.com/v7labs/darwin-py
* https://docs.v7labs.com/docs/loading-a-dataset-in-python
* https://github.com/v7labs/darwin-py/blob/master/darwin/torch/dataset.py

In [2]:
from pathlib import Path
import pyprojroot
dir_proj = pyprojroot.here()
print("Project directory:", dir_proj)

Project directory: /home/ck37/projects/ecig-vaping


In [3]:
# Review all datasets
!darwin dataset remote

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ Name                                           ┃ Item Count ┃ Complete Items ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ uscharvardecig/ecig-instagram-objection-detec… │       4645 │          24.1% │
└────────────────────────────────────────────────┴────────────┴────────────────┘


In [5]:
#team_name = "annotatevape" # Old account, pre March 2023
team_name = "uscharvardecig" # Academic account, starting March 2023

#dataset_name = "training-data" # Old dataset name, pre March 2023
dataset_name = "ecig-instagram-objection-detection" # New dataset name, starting March 2023

#dataset_id = "annotatevape/training-data"
#dataset_id = "annotatevape/training-data:top-5-classes-v2"
#dataset_id= "annotatevape/training-data:top-5-classes-v4" # Dec. 12, 2021
#dataset_id= "annotatevape/training-data:many-in-review" # Jan. 10, 2021
#dataset_id= "annotatevape/training-data:sample1-complete-570" # Jan. 13, 2021
#dataset_version = "reviewed-777" # Jan. 17, 2022
#dataset_version = "1030-complete-ck-tweaked" # Feb. 7, 2022
#dataset_version = "complete-in-review-1434-may-2nd" # May 2, 2022
#dataset_version = "complete-or-in-review-2313" # May 16, 2022
#dataset_version = "complete-or-in-review-2493" # May 23, 2022
#dataset_version = "complete-in-review-2519" # Aug. 4, 2022
#dataset_version = "all-data-3258-items" # Oct. 24, 2022
#dataset_version = "all-data-3287-items" # Oct. 24, 2022

dataset_version = "2023-03-25-4632"

dataset_id = team_name + "/" + dataset_name + ":" + dataset_version
print("Dataset id:", dataset_id)

Dataset id: uscharvardecig/ecig-instagram-objection-detection:2023-03-25-4632


In [6]:
# Review locally downloaded datasets
!darwin dataset local

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Name                                    ┃ Image Count ┃ Sync Date ┃     Size ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━┩
│ uscharvardecig/ecig-instagram-objectio… │        4348 │     today │ 656.5 MB │
└─────────────────────────────────────────┴─────────────┴───────────┴──────────┘


In [7]:
# Pull necessary dataset
!darwin dataset pull {dataset_id}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━0m
Dataset uscharvardecig/ecig-instagram-objection-detection:2023-03-25-4632 downloaded at /home/ck37/.darwin/datasets/uscharvardecig/ecig-instagram-objection-detection. 


In [9]:
# Pull dataset and divide into training, val, and test
# --stratified-types bounding_box
!darwin dataset split {dataset_id} --seed 1 --val-percentage .10 --test-percentage .20 

Partition lists saved at /home/ck37/.darwin/datasets/uscharvardecig/ecig-instagram-objection-detection/releases/2023-03-25-4632/lists/3045_434_869_s1


In [11]:
#!ls /home/ck37/.darwin/datasets/annotatevape/training-data/releases/top-5-classes-v2/lists/split_v0_t0
#!ls /home/ck37/.darwin/datasets/annotatevape/training-data/releases/sample1-complete-570/lists/default
!ls /home/ck37/.darwin/datasets/{team_name}/{dataset_name}/releases/{dataset_version}/lists/default

random_test.txt   stratified_bounding_box_test.txt   stratified_tag_test.txt
random_train.txt  stratified_bounding_box_train.txt  stratified_tag_train.txt
random_val.txt	  stratified_bounding_box_val.txt    stratified_tag_val.txt


In [12]:
from darwin.torch import get_dataset
dataset = get_dataset(dataset_id, dataset_type="object-detection")
print(dataset)

print(dataset.classes)

/home/ck37/.conda/envs/mmdet/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ValueError: Annotation (/home/ck37/.darwin/datasets/uscharvardecig/ecig-instagram-objection-detection/releases/2023-03-25-4632/annotations/2021-02-17 21.47.29 2511607846950066850_186439710.json) does not have a corresponding image

## Convert v7 to COCO format

In [13]:
dir_base = Path("/home/ck37/.darwin/datasets/" + team_name + "/" + dataset_name + "/releases/")
dir_labels = dir_base / dataset_version

!ls {dir_labels}

annotations  lists


In [14]:
# These are our train/val/test splits:
dir_lists = dir_labels / "lists"
print("Lists:")
!ls {dir_lists}

print("Splits:")
dir_split = dir_labels / "lists/default"
!ls {dir_split}

Lists:
3045_434_869_s1  classes_bounding_box.txt  classes_tag.txt  default
Splits:
random_test.txt   stratified_bounding_box_test.txt   stratified_tag_test.txt
random_train.txt  stratified_bounding_box_train.txt  stratified_tag_train.txt
random_val.txt	  stratified_bounding_box_val.txt    stratified_tag_val.txt


In [15]:
# Import class names
filename = dir_lists / "classes_bounding_box.txt"
classes = open(filename, "r").read().splitlines()
print(f'Found {len(classes)} classes.')
print(classes)

# Convert to coco category format
categories = [{'supercategory': None, 'id': i, 'name': class_i} for i, class_i in enumerate(classes)]
print("Categories:", categories)

Found 12 classes.
['box', 'e-cigarette brand name', 'e-juice', 'e-juice flavor', 'e-liquid', 'flavorname', 'mod', 'pod', 'smoke cloud', 'synthetic nicotine label', 'warning label nicotine', 'warning label tobacco-free']
Categories: [{'supercategory': None, 'id': 0, 'name': 'box'}, {'supercategory': None, 'id': 1, 'name': 'e-cigarette brand name'}, {'supercategory': None, 'id': 2, 'name': 'e-juice'}, {'supercategory': None, 'id': 3, 'name': 'e-juice flavor'}, {'supercategory': None, 'id': 4, 'name': 'e-liquid'}, {'supercategory': None, 'id': 5, 'name': 'flavorname'}, {'supercategory': None, 'id': 6, 'name': 'mod'}, {'supercategory': None, 'id': 7, 'name': 'pod'}, {'supercategory': None, 'id': 8, 'name': 'smoke cloud'}, {'supercategory': None, 'id': 9, 'name': 'synthetic nicotine label'}, {'supercategory': None, 'id': 10, 'name': 'warning label nicotine'}, {'supercategory': None, 'id': 11, 'name': 'warning label tobacco-free'}]


## Generate output jsons

In [23]:
from pycocotools.coco import COCO
import os
import json, re, csv
import cv2
import numpy as np
import pandas as pd

def v7_annotations_to_coco_json(input_file, output_json,
                                img_dir = dir_proj / "data-raw/training-data/images"):
    filename = dir_split / input_file
    image_files = open(filename, "r").read().splitlines()
    print(f'Found {len(image_files)} images.')
    print(image_files[:5])

    # Import annotation jsons.
    dir_json = dir_labels / "annotations"

    # Bounding box list of dictionaries.
    annotations = []
    image_dicts = []
    ann_counter = 0
    channel_stats = []
    
    # Classification list of dictionaries.
    class_annotations = []

    for i, image_i in enumerate(image_files):
        json_file = dir_json / str(image_i + ".json")
        if os.path.exists(json_file):
            #annotations.append(json_file)

            # Import the json file
            ann = json.load(open(json_file, 'r'))
            
            # TODO: determine the file extension automatically so that it works for png or jpeg.
            image_filename = image_i + '.jpg'
            
            image_path = img_dir / image_filename
            if not image_path.exists():
                print("Could not find image, skipping.", image_path)
                continue


            # Extract the image info
            image_dict = {
                'width': ann['image']['width'],
                'height': ann['image']['height'],
                'id': image_i,
                'file_name': image_filename
            }
            image_dicts.append(image_dict)
            
            # Load image and extract channel means and stddevs.
            img = cv2.imread(str(image_path))
            b = img[:,:,0]
            g = img[:,:,1]
            r = img[:,:,2]
            channel_stats.append({
                'mean_r': np.mean(r),
                'mean_g': np.mean(g),
                'mean_b': np.mean(b),
                'std_r': np.std(r),
                'std_g': np.std(g),
                'std_b': np.std(b)
            })
            
            if 'annotations' in ann.keys():
                #print('Annotations:', ann['annotations'])
                for ann_i in ann['annotations']:
                    #######################
                    # Handle bounding boxes
                    if 'bounding_box' in ann_i.keys():
                        bbox = ann_i['bounding_box']
                        area = bbox['h'] * bbox['w']

                        # elements should be x, y, width, height
                        bbox_new = [bbox['x'], bbox['y'], bbox['w'], bbox['h']]

                        ann_dict = {
                            'area': area,
                            'bbox': bbox_new,
                            'iscrowd': 0,
                            'segmentation': [],
                            'image_id': image_i,
                            'id': ann_counter,
                            # TODO: confirm that this gives the same index as the categories list above.
                            'category_id': classes.index(ann_i['name'])
                        }

                        ann_counter = ann_counter + 1
                        annotations.append(ann_dict)
                        
                    #######################
                    # Handle nonvaping tag
                    class_img = {
                        'file_name': image_filename
                    }
                    
                    if 'name' in ann_i.keys() and ann_i['name'] == 'nonvaping':
                        # Nonvaping image
                        class_img['vaping'] = 0
                    else:
                        # Vaping image
                        class_img['vaping'] = 1
                        
                    class_annotations.append(class_img)

        else:
            print(f'Could not find: {json_file}')
        
    ###############
    # Bounding box export.
    data_coco = {}

    data_coco["images"] = image_dicts
    data_coco["categories"] = categories
    data_coco["annotations"] = annotations
    
    # Create bounding box export.
    # TODO: check that output_json can be opened.
    json.dump(data_coco, open(output_json, "w"), indent=4)
        
    # See if the newly created json can be imported
    coco = COCO(output_json)
    # See if we can determine the category ids for our predefined classes
    cat_ids = coco.getCatIds(classes)
    print("Categories IDs confirmed:", cat_ids)
    
    ##################
    # Image classification export.
    # Add -class just before .json in the output filename, and change to .txt
    class_output_json = re.sub(r'(.json)$', r'-class.txt', str(output_json))
    with open(class_output_json, 'w') as csvfile:
        # openmmlab expects fields to be separated by spaces rather than commas.
        writer = csv.DictWriter(csvfile, delimiter = " ", fieldnames = class_annotations[1].keys())
        writer.writerows(class_annotations)
        
        
    # Calculate channel stats.
    channel_df = pd.DataFrame(channel_stats)
    return(channel_df)

In [24]:
dir_results = dir_proj / "data/annotations/"

print("Training set creation")
train_df = v7_annotations_to_coco_json("stratified_bounding_box_train.txt",
                            dir_results / "v7-coco-train.json")

# Actually we don't use these; we want to stick with the values used during pretraining.
print(train_df.mean())

print("\nValidation set creation")
v7_annotations_to_coco_json("stratified_bounding_box_val.txt",
                            dir_results / "v7-coco-val.json")

print("\nTest set creation")
v7_annotations_to_coco_json("stratified_bounding_box_test.txt",
                            dir_results / "v7-coco-test.json");

Training set creation
Found 3045 images.
['twisteliquids==2021-06-04_19-02-11_UTC', '2022-04-18_17-09-56_UTC', 'mustaqimahjati==2020-06-07_14-02-26_UTC', '20210226-drewdirps-0120-p1', '2023-01-18 01.56.55 3018353646592888436_6848110601']
Could not find image, skipping. /home/ck37/projects/ecig-vaping/data-raw/training-data/images/000000005.jpg
Could not find image, skipping. /home/ck37/projects/ecig-vaping/data-raw/training-data/images/2022-01-08 01.58.06 2746563148290867850_207580568.jpg
Could not find image, skipping. /home/ck37/projects/ecig-vaping/data-raw/training-data/images/000000014.jpg
Could not find image, skipping. /home/ck37/projects/ecig-vaping/data-raw/training-data/images/000000009.jpg
Could not find image, skipping. /home/ck37/projects/ecig-vaping/data-raw/training-data/images/2021-04-05 07.12.43 2545203421029675085_3231772376.jpg
Could not find image, skipping. /home/ck37/projects/ecig-vaping/data-raw/training-data/images/2021-12-16 02.15.08 2729901684211961245_3963274

## Image classification

In [38]:
from darwin.torch import get_dataset
dataset = get_dataset(dataset_id, dataset_type="classification")
print(dataset)
# Make sure that "exclude" is not one of the classes.
print(dataset.classes)

ClassificationDataset():
  Root: /home/ck37/.darwin/datasets/annotatevape/training-data
  Number of images: 2487
  Number of classes: 1
['nonvaping']


In [39]:
dataset

In [40]:

for i, item in enumerate(dataset):
    info = dataset.get_img_info(i)
    print(i, info)

0 {'width': 720, 'height': 404, 'original_filename': '20190101-billytricks-0770-p1.jpg', 'filename': '20190101-billytricks-0770-p1.jpg', 'url': 'https://darwin.v7labs.com/api/images/263263371/original', 'thumbnail_url': 'https://darwin.v7labs.com/api/images/263263371/thumbnail', 'path': '/', 'workview_url': 'https://darwin.v7labs.com/workview?dataset=387034&image=855'}
1 {'width': 720, 'height': 404, 'original_filename': '20190101-billytricks-0771-p1.jpg', 'filename': '20190101-billytricks-0771-p1.jpg', 'url': 'https://darwin.v7labs.com/api/images/263263230/original', 'thumbnail_url': 'https://darwin.v7labs.com/api/images/263263230/thumbnail', 'path': '/', 'workview_url': 'https://darwin.v7labs.com/workview?dataset=387034&image=714'}
2 {'width': 720, 'height': 404, 'original_filename': '20190103-billytricks-0772-p1.jpg', 'filename': '20190103-billytricks-0772-p1.jpg', 'url': 'https://darwin.v7labs.com/api/images/263263258/original', 'thumbnail_url': 'https://darwin.v7labs.com/api/image

AssertionError: No tags were found for index=13

In [18]:
dataset.annotations

AttributeError: 'ClassificationDataset' object has no attribute 'annotations'